In [12]:
!cd /home/user/Desktop/Programming/Keras/Mask_R_CNN/Mask_RCNN
!pwd

/home/user/Desktop/Programming/Keras/Mask_R_CNN/Mask_RCNN/samples/form_login


In [2]:
import socket

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/user/Desktop/Programming/Keras/Mask_R_CNN/mask_rcnn_venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-4996ee3d8d09>", line 1, in <module>
    from google.colab import drive
ModuleNotFoundError: No module named 'google.colab'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/user/Desktop/Programming/Keras/Mask_R_CNN/mask_rcnn_venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/user/Desktop/Programming/Keras/Mask_R_CNN/mask_rcnn_venv/lib/python3.6/site-packages/IPython/core/ul

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/mask_rcnn')

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import skimage.draw
import json
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("/content/gdrive/My Drive/mask_rcnn")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Mask R-CNN - Train on Shapes Dataset


This notebook shows how to train Mask R-CNN on your own dataset. To keep things simple we use a synthetic dataset of shapes (squares, triangles, and circles) which enables fast training. You'd still need a GPU, though, because the network backbone is a Resnet101, which would be too slow to train on a CPU. On a GPU, you can start to get okay-ish results in a few minutes, and good results in less than an hour.

The code of the *Shapes* dataset is included below. It generates images on the fly, so it doesn't require downloading any data. And it can generate images of any size, so we pick a small image size to train faster. 

## Configurations

In [ ]:
class LoginConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
        # Give the configuration a recognizable name
    NAME = "form_login"

    # Number of classes (including background)
    NUM_CLASSES = 1 + 7  # Background + form_login

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
config = LoginConfig()
config.display()

## Notebook Preferences

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Dataset

Create a synthetic dataset

Extend the Dataset class and add a method to load the shapes dataset, `load_shapes()`, and override the following methods:

* load_image()
* load_mask()
* image_reference()

In [ ]:
class Form_loginDataset(utils.Dataset):

    def load_form_login(self, dataset_dir, subset):
        """Load a subset of the Form_login dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have five classes to add.
        self.add_class("form_login", 1, "input")
        self.add_class("form_login", 2, "text")
        self.add_class("form_login", 3, "button")
        self.add_class("form_login", 4, "checkbox")
        self.add_class("form_login", 5, "radiobutton")
        self.add_class("form_login", 6, "image")
        self.add_class("form_login", 7, "select")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'rectangle'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join(dataset_dir, "via_export_json.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the rectangles that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            
            # The type of the object is list
            '''
            if type(a['regions']) is dict:
                rectangles = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                rectangles = [r['shape_attributes'] for r in a['regions']] 
            '''
            rectangles = [r['shape_attributes'] for r in a['regions']] 
            for rectangle in rectangles:
                print(rectangle)
            # load_mask() needs the image size to convert rectangles to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.

            objects = [r['region_attributes'] for r in a['regions']]

            '''
            # Check the presense of the HTML elements types
            classids = [(n['HTML element']) for n in objects]
            for classid in classids:
                print(classid.keys())

            class_ids = [int(n['HTML element']) for n in objects]
            '''

            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "form_login",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                rectangles=rectangles,
                objects=objects)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a form_login dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "form_login":
            return super(self.__class__, self).load_mask(image_id)

        class_names = image_info['objects']

        # Convert rectangles to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["rectangles"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["rectangles"]):
            # print(i)
            # print(p)
            # Get indexes of pixels inside the rectangle and set them to 1
            # rr, cc = skimage.draw.rectangle(p['all_points_y'], p['all_points_x'])
            # https://scikit-image.org/docs/dev/api/skimage.draw.html#skimage.draw.rectangle
            #img = np.zeros((int(p['width']*2), int(p['height']*2)), dtype=np.uint8)
            
            #start = (p['x'], p['y'] - p['height'])
            #extent = (p['width'], p['height'])
            
            start = (p['y'], p['x'])
            extent = (p['height'], p['width'])
            
            
            # rr, cc = skimage.draw.rectangle(start, extent=extent, shape=img.shape)
            rr, cc = skimage.draw.rectangle(start, extent=extent)            
            # print(rr)
            # print(cc)
            mask[rr, cc, i] = 1
        # Assign class_ids by reading class_names
        class_ids = np.zeros([len(info["rectangles"])])
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

        # In the form_login dataset, pictures are labeled with name 'input' and 'button' representing input and button.
        for i, p in enumerate(class_names):
            # print('HTML element=', p['HTML element'])
            # print("i=",i)
            # temp = p['HTML element']
            # for key in temp:
            #     print('temp=', temp)
            #     print(key)
            # "name" is the attributes name decided when labeling, etc. 'region_attributes': {name:'a'} // "region_attributes":{"HTML element":{"input":true}}}
            '''
            if p['HTML element'] == 'input':
                class_ids[i] = 1

            elif p['HTML element'] == 'button':
                class_ids[i] = 2

            elif p['HTML element'] == 'hyperlink':
                class_ids[i] = 3

            elif p['HTML element'] == 'text':
                class_ids[i] = 4

            elif p['HTML element'] == 'select':
                class_ids[i] = 5
            '''

            if 'input' in p['type']:
                class_ids[i] = 1

            elif 'text' in p['type']:
                class_ids[i] = 2

            elif 'button' in p['type']:
                class_ids[i] = 3

            elif 'checkbox' in p['type']:
                class_ids[i] = 4

            elif 'radiobutton' in p['type']:
                class_ids[i] = 5

            elif 'image' in p['type']:
                class_ids[i] = 6

            elif 'select' in p['type']:
                class_ids[i] = 7


        # assert code here to extend to other labels

        # Should return multiple classes ids
        class_ids = np.array(class_ids, dtype=np.int32)

        # class_ids = class_ids.astype(int)

        return mask.astype(np.bool), class_ids

    

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "form_login":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [ ]:
# Training dataset
dataset_train = Form_loginDataset()
dataset_train.load_form_login('gdrive/My Drive/Colab Notebooks/dataset','train')
dataset_train.prepare()

# Validation dataset
dataset_val = Form_loginDataset()
dataset_val.load_form_login('gdrive/My Drive/Colab Notebooks/dataset','train')
dataset_val.prepare()

In [ ]:
!cat gdrive/My\ Drive/Colab\ Notebooks/dataset/train/via_export_json.json

In [ ]:
!ls gdrive/My\ Drive/Colab\ Notebooks

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

## Create Model

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=4, 
            layers='heads')

In [ ]:
!ls

In [ ]:
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
model.keras_model.save_weights(model_path)

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=7, 
            layers="all")

In [ ]:
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
model.keras_model.save_weights(model_path)

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.load_weights(model_path, by_name=True)
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=3, 
            layers="all")

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
 model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
 model.keras_model.save_weights(model_path)

## Detection

In [ ]:
class InferenceConfig(LoginConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [ ]:
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

## Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))